In [3]:
import pandas as pd

adult_census = pd.read_csv('adult-census.csv')
target_name = 'class'
data = adult_census.drop(columns=[target_name, 'education-num'])
target = adult_census[target_name]

In [4]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_selector, make_column_transformer

categorical_encoder = OrdinalEncoder(
    handle_unknown='use_encoded_value', unknown_value=-1
)
preprocessor = make_column_transformer(
    (categorical_encoder, make_column_selector(dtype_include=object)),
    remainder='passthrough'
)

In [5]:
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier

tree = make_pipeline(preprocessor, DecisionTreeClassifier(random_state=0))

In [6]:
from sklearn.model_selection import cross_val_score

scores_tree = cross_val_score(tree, data, target)
print("DecisionTreeClassifier: "
      f"{scores_tree.mean():.3f} +/- {scores_tree.std():.3f}")

DecisionTreeClassifier: 0.820 +/- 0.006


In [7]:
from sklearn.ensemble import BaggingClassifier

bagged_trees = make_pipeline(preprocessor, BaggingClassifier(estimator=DecisionTreeClassifier(random_state=0),
                             n_estimators=50,
                             n_jobs=2,
                             random_state=0))
bagged_trees

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ordinalencoder',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000023B0A02BEE0>)])),
                ('baggingclassifier',
                 BaggingClassifier(estimator=DecisionTreeClassifier(random_state=0),
                                   n_estimators=50, n_jobs=2,
                                   random_state=0))])

In [8]:
scores_bagged_trees = cross_val_score(bagged_trees, data, target) 

print(
    "Bagged decision tree classifier: "
    f"{scores_bagged_trees.mean():.3f} +/- {scores_bagged_trees.std():.3f}"
)

Bagged decision tree classifier: 0.846 +/- 0.005


In [9]:
from sklearn.ensemble import RandomForestClassifier

random_forest = make_pipeline(preprocessor, RandomForestClassifier(n_estimators=50, n_jobs=2, random_state=0))#no estimator needs to be specified because it has to be decision tree

In [10]:
scores_random_forest = cross_val_score(random_forest, data, target)
print("RandomForestClassifier: "
      f"{scores_random_forest.mean():.3f} +/- {scores_random_forest.std():.3f}")

RandomForestClassifier: 0.851 +/- 0.004
